In [83]:
import pandas as pd
import os
import numpy as np
from config import apikey
import gmaps

In [84]:
bcross_df = pd.read_csv('Border_Crossing_Entry_Data.csv')
bcross_df.head()

,Port Name,State,Port Code,Border,Date,Measure,Value,Location
0,Calexico East,California,2507,US-Mexico Border,03/01/2019 12:00:00 AM,Trucks,34447,POINT (-115.48433000000001 32.67524)
1,Van Buren,Maine,108,US-Canada Border,03/01/2019 12:00:00 AM,Rail Containers Full,428,POINT (-67.94271 47.16207)
2,Otay Mesa,California,2506,US-Mexico Border,03/01/2019 12:00:00 AM,Trucks,81217,POINT (-117.05333 32.57333)
3,Nogales,Arizona,2604,US-Mexico Border,03/01/2019 12:00:00 AM,Trains,62,POINT (-110.93361 31.340279999999996)
4,Trout River,New York,715,US-Canada Border,03/01/2019 12:00:00 AM,Personal Vehicle Passengers,16377,POINT (-73.44253 44.990010000000005)


In [86]:


bcross_df['new_loc'] = bcross_df['Location'].str.split(" ")
#[.split(" ") ]

#bcross_df['new_loc'][0][2]
bcross_df.loc[:, 'lng'] = bcross_df.new_loc.map(lambda x: x[1][1:])
bcross_df.loc[:, 'lat'] = bcross_df.new_loc.map(lambda x: x[2][:-1])


 
#coordinates = [str.split(" ") for str in locs]
bcross_df['port'] = bcross_df['Port Name']
names = bcross_df.port.unique()
lat = bcross_df.lat.unique()
lng = bcross_df.lng.unique()
#gah = pd.DataFrame({"Port": names ,"Latitude": lat, "Longitude": lng})


In [89]:
transport_information = bcross_df.drop(["Port Code","Border","Location","Date","State","new_loc"], axis=1)
transport_information = transport_information.loc[transport_information['Value'] !=0]
nonhumans = ["Trucks", "Buses", "Personal Vehicles"]
transport_info2 = transport_information[transport_information.Measure.isin(nonhumans)]
by_port_name = transport_info2.groupby(['Port Name','lat','lng', 'Measure'])['Value'].sum()



transport_info_sorted = transport_info2.groupby(['Port Name'])['Value'].sum()
transport_info_sorted2 = transport_info_sorted.sort_values(ascending=False)
coordsname = pd.DataFrame({"Port Name":by_port_name['Port Name'], "Latutude":by_port_name['lat'], 
                           "Longitude":by_port_name['lng']})
transport_info_sorted

KeyError: 'Port Name'

AttributeError: 'Series' object has no attribute 'Measure'

In [27]:
gmaps.configure(api_key=apikey)
locations = by_port_name[["lat", "lng"]].astype(float)
rating = by_port_name["Value"].astype(float)
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)
fig

KeyError: 'Value'